# Postprocess predictions and create a submission

## Configure

In [26]:
#@title Submission notes and params { run: "auto" }
version_notes = "BBox reduction" #@param {type:"string"}

reduce_method = "weighted_boxes_fusion" #@param ["None", "nms", "soft_nms", "non_maximum_weighted", "weighted_boxes_fusion"]

iou_threshold = 0.6 #@param {type:"slider", min:0, max:1, step:0.01}
skip_box_threshold = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
sigma = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

reduce_filter = False #@param {type:"boolean"}

binary_abnormal_threshold = 0.05 #@param {type:"slider", min:0, max:1, step:0.01}
binary_normal_threshold = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}
assert(binary_abnormal_threshold <= binary_normal_threshold)

SUBMIT = True #@param {type:"boolean"}

In [2]:
INPUTS_YOLO_VERSIONS = {
    5: {
        "version_notes": "YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15",
        # "path": f"v{version}",
        # "enabled": version in yolo_versions_include,
    },
    6: { "version_notes": "WBF preproc, YOLOv5x, 50 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15", },
    7: { "version_notes": "WBF preproc, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1", },
    8: { "version_notes": "WBF preproc with IoU 0.4, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15, remove (most) augs", },
    9: { "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs", },
    10: { "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 25 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs", },
    11: { "version_notes": "Remove (most) augs --- WBF preproc IoU: 0.8, version: yolov5l, epochs: 30, valid split: 20.0%, image size: 1024, detect IoU: 0.8, detect conf: 0.05", },
    # 5 Folds of the same model
    12: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 0/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    13: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 1/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    14: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 2/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    15: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 3/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    16: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 4/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    # version 17 and 18 are identical.
    # 5 Folds, WBF at 0.4
    18: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: exp_yolov5x_50epochs_WBFat0.4iou_640image_detect0.6iou0.01conf_fold0of5", },
    19: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    20: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    21: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    22: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    #
    23: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.65; detect conf: 0.001; exp: yolov5x_40epochs_640image_fold0of5_WBFat0.4iou_det0.65iou0.001conf", },
    24: { "version_notes": "Rad weight as number of labels, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # 5 Folds for outliers filter with Gaussian model
    25: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    26: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    27: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    28: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    29: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # version 30 did very poorly in CV
    31: { "version_notes": "Filter outliers (bbox size) with quantiles, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.8; detect conf: 0.01; exp: yolov5x_outliers_quantiles_40epochs_640image_fold0of5_WBFat0.4iou_det0.8iou0.01conf", },
    32: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 100; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_100epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # versions 33-37 are versions 25-29 with TTA
    33: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    34: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    35: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    36: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    37: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    #
    38: { "version_notes": "Filter outliers (bbox size) with Gaussian model, use focal loss at 1.5, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 60; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_fl_60epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    # Use GroupKFold
    39: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    40: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    41: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold2of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    42: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    43: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    # Versions 44-48 are versions 39-43 without TTA
    44: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    45: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    46: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold2of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    47: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    48: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    #
    49: { "version_notes": "Filter outliers (bbox size) with Gaussian model, fix aort.enl. vs cardiomeg., Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 704; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_40epochs_704image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    50: { "version_notes": "Filter outliers (bbox size) with Gaussian model, fix aort.enl. vs cardiomeg., Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 704; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_40epochs_704image_fold1of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
}
# The dictionary shouldn't allow duplicate keys anyway.
assert len(set(INPUTS_YOLO_VERSIONS.keys())) == len(INPUTS_YOLO_VERSIONS), "Version dictionary has repeated keys"
for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    assert all(list(key_name in version_data for key_name in ["version_notes"])), f"Not all keys present in version {yolo_version} definitions"
    assert version_data["version_notes"], f"Version notes not added for version {yolo_version}"

In [3]:
#@title Select models versions { run: "auto" }

binary_version = "7: EfficientNetB3, include probabilities (for later double thresholding); image size: (640, 640); threshold: 0.8" #@param ["2: Try 1024 resolution", "3: EffNet, 1024 resolution, threshold 0.8", "4: EffNet, 1024 resolution, threshold 0.8", "5: EffNet, 512 resolution, threshold 0.8", "6: EfficientNetB3, include probabilities (for later double thresholding); image size: (1024, 1024); threshold: 0.8", "7: EfficientNetB3, include probabilities (for later double thresholding); image size: (640, 640); threshold: 0.8"]
binary_notes = binary_version.split(":")[1]
binary_version = int(binary_version.split(":")[0])


yolo_versions_include = "18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 44, 45, 46, 47, 48, 32, 49, 50" #@param {type:"string"}
yolo_versions_include = list(map(int, yolo_versions_include.split(","))) if yolo_versions_include else []

yolo_versions_weighted = False #@param {type:"boolean"}
yolo_versions_weights = { 18: 0.183, 19: 0.183, 20: 0.183, 21: 0.183, 22: 0.203, 25: 0.203, 26: 0.203, 27: 0.203, 28: 0.203, 29: 0.203, 44: 0.173, 45: 0.173, 46: 0.173, 47: 0.173, 48: 0.173, 32: 0.204, } #@param

# Check that versions are distinct (no need to ensemble the same model)
assert len(yolo_versions_include) == len(set(yolo_versions_include)), "Versions to include have repeated entries"
assert all(version in INPUTS_YOLO_VERSIONS for version in yolo_versions_include), "Unknown version numbers present in version to include"
print(yolo_versions_include)

if yolo_versions_weighted:
    assert all(version in yolo_versions_weights for version in yolo_versions_include), "No weight defined for some of the versions"
    print(yolo_versions_weights)

[18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 44, 45, 46, 47, 48, 32, 49, 50]


In [4]:
for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    version_data["path"] = f"v{yolo_version}"
    version_data["enabled"] = yolo_version in yolo_versions_include

# Filter only enabled.
INPUTS_YOLO_VERSIONS = {yolo_version: version_data for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items() if version_data["enabled"]}

## Setup

In [5]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, WORK_FOLDER
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/output', '/kaggle/working', '/kaggle/input']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [6]:
from pathlib import Path

from IPython.display import clear_output

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER_ORIGINAL_PNG = WORK_FOLDER / "vinbigdata-chest-xray-original-png"
INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_BINARY = WORK_FOLDER / "vbdbinary"

for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    version_data["path"] = INPUT_FOLDER_YOLO_OUT / version_data["path"]

print(f"Using the following YOLO versions:")
print("\n".join(f"{yolo_version}: {version_data}" for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items()))

Using the following YOLO versions:
18: {'version_notes': 'Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: exp_yolov5x_50epochs_WBFat0.4iou_640image_detect0.6iou0.01conf_fold0of5', 'path': PosixPath('/kaggle/working/vbdyolo-out/v18'), 'enabled': True}
19: {'version_notes': 'Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf', 'path': PosixPath('/kaggle/working/vbdyolo-out/v19'), 'enabled': True}
20: {'version_notes': 'Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image

## Get data from Kaggle

In [7]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@fix-datasets-download-file-unzip
clear_output()

In [8]:
!kaggle datasets download corochann/vinbigdata-chest-xray-original-png -f test_meta.csv -p {INPUT_FOLDER_ORIGINAL_PNG} --unzip

  0% 0.00/126k [00:00<?, ?B/s]
100% 126k/126k [00:00<00:00, 55.2MB/s]


In [9]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@add-datasets-download-version
clear_output()

In [10]:
for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    !rm -rf {version_data["path"]}
    !kaggle datasets download "witalia/vbdyolo-out-newest" -v {yolo_version} -p {version_data["path"]} --unzip --force

 98% 306M/311M [00:02<00:00, 143MB/s]
100% 311M/311M [00:02<00:00, 160MB/s]
100% 310M/311M [00:03<00:00, 93.7MB/s]
100% 311M/311M [00:03<00:00, 99.9MB/s]
 96% 299M/311M [00:03<00:00, 111MB/s] 
100% 311M/311M [00:03<00:00, 102MB/s]
 93% 290M/311M [00:01<00:00, 166MB/s]
100% 311M/311M [00:01<00:00, 165MB/s]
 95% 294M/311M [00:01<00:00, 180MB/s]
100% 311M/311M [00:01<00:00, 190MB/s]
 98% 303M/311M [00:01<00:00, 167MB/s]
100% 311M/311M [00:01<00:00, 173MB/s]
 98% 306M/311M [00:01<00:00, 224MB/s]
100% 311M/311M [00:01<00:00, 223MB/s]
 98% 303M/311M [00:01<00:00, 157MB/s]
100% 311M/311M [00:01<00:00, 173MB/s]
 99% 307M/311M [00:01<00:00, 177MB/s]
100% 311M/311M [00:01<00:00, 181MB/s]
 96% 298M/311M [00:01<00:00, 175MB/s]
100% 311M/311M [00:01<00:00, 196MB/s]
 96% 300M/311M [00:01<00:00, 175MB/s]
100% 311M/311M [00:01<00:00, 196MB/s]
100% 311M/311M [00:02<00:00, 131MB/s]
100% 311M/311M [00:02<00:00, 156MB/s]
 95% 296M/311M [00:01<00:00, 237MB/s]
100% 311M/311M [00:01<00:00, 232MB/s]
 99% 306M

In [11]:
!rm -rf {INPUT_FOLDER_BINARY}
!kaggle datasets download "witalia/vbdbinary" -v {binary_version} -p {INPUT_FOLDER_BINARY} --unzip --force

 92% 97.0M/106M [00:00<00:00, 141MB/s]
100% 106M/106M [00:00<00:00, 190MB/s] 


## Process YOLO output

### BBox reduction (potentially from multiple models)

In [12]:
!pip install ensemble-boxes
clear_output()

In [13]:
from ensemble_boxes import *
from typing import List

def reduce_boxes(image_labels_list: List[pd.DataFrame], weights=None, iou_thr=iou_threshold, skip_box_thr=skip_box_threshold, sigma=sigma, method=None):
    label_empty = [len(df) == 0 for df in image_labels_list]

    if len(image_labels_list) == 0 or all(label_empty):
        # If there's nothing to fuse, return empty.
        return pd.DataFrame(columns=["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Ensemble_boxes doesn't do well with empty inputs, so skip those.
    labels = [image_labels["class_id"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    scores = [image_labels["conf"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    boxes = [image_labels[["x_min", "y_min", "x_max", "y_max"]].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    if weights is not None:
        weights = [weight for weight, empty in zip(weights, label_empty) if not empty]

    if method == non_maximum_weighted or method == weighted_boxes_fusion:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr, weights=weights)
    elif method == soft_nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, thresh=skip_box_thr, sigma=sigma, weights=weights)
    elif method == nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, weights=weights)
    else:
        if len(labels) > 1:
            raise ValueError("Cannot fuse more than 1 model without a fusing method")
        [boxes], [scores], [labels] = boxes, scores, labels

    reduced_labels = pd.DataFrame().reindex(list(range(len(labels))))
    reduced_labels["class_id"] = np.array(labels).astype(np.int32)
    reduced_labels["conf"] = scores
    reduced_labels[["x_min", "y_min", "x_max", "y_max"]] = boxes

    return reduced_labels


def reduce_max_x_per_class(image_labels: pd.DataFrame):
    max_counts = [
        ([0, 3], 3),  # Aortic Enlargement and Cardiomegaly
        ([11, 13], 25),  # Pleaural thickening and Pulmonary fibrosis
        (list(set(range(14)) - set([0, 3, 11, 13])), 50)
    ]

    result_df = image_labels

    for class_ids, max_count in max_counts:
        # Aortic Enlargement and Cardiomegaly should be max one on the image.
        one_class_mask = result_df["class_id"].isin(class_ids)

        # Take most confident boxes in each class.
        result_df_one_class = result_df[one_class_mask].sort_values("conf", ascending=False).groupby("class_id").head(max_count)

        result_df = result_df[~one_class_mask]
        result_df = result_df.append(result_df_one_class)

    return result_df


def reduce_by_location_boundaries(image_labels: pd.DataFrame):
    image_labels["x_mid"] = (image_labels["x_min"] + image_labels["x_max"]) / 2
    image_labels["y_mid"] = (image_labels["y_min"] + image_labels["y_max"]) / 2

    valid_labels_mask = image_labels["class_id"] == image_labels["class_id"]

    # Remove those that are in wrong location for Aortic Enlargement
    aort_enlarg_wrong_loc = (image_labels["class_id"] == 0) & \
        ((image_labels["x_mid"] < 0.3) | (image_labels["x_mid"] > 0.8) | (image_labels["y_mid"] > 0.6))
    valid_labels_mask = valid_labels_mask & (~aort_enlarg_wrong_loc)

    # Remove those that are in wrong location for 
    cardiomeg_wrong_loc = (image_labels["class_id"] == 3) & \
        ((image_labels["x_mid"] < 0.4) | (image_labels["x_mid"] > 0.9) | (image_labels["y_mid"] < 0.4))
    valid_labels_mask = valid_labels_mask & (~cardiomeg_wrong_loc)

    # Any bbox shouldn't be very close to border (except for "Other lesion")
    any_bbox_wrong_loc = (image_labels["class_id"] != 9) & \
        ((image_labels["x_mid"] < 0.03) | (image_labels["x_mid"] > 0.97) | \
            (image_labels["y_mid"] < 0.1) | (image_labels["y_mid"] > 0.9))
    valid_labels_mask = valid_labels_mask & (~any_bbox_wrong_loc)

    # Remove the temporary columns.
    image_labels = image_labels.drop(columns=["x_mid", "y_mid"])

    return image_labels[valid_labels_mask]

### Read and transform labels

In [14]:
def read_prediction_labels(filename: Path):
    yolo_columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]
    if filename.exists():
        labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
        labels.columns = yolo_columns
    else:
        labels: pd.DataFrame = pd.DataFrame(columns=yolo_columns)

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = (labels["x_centre"] - labels["bw"] / 2).clip(0, 1)
    labels["y_min"] = (labels["y_centre"] - labels["bh"] / 2).clip(0, 1)
    labels["x_max"] = (labels["x_centre"] + labels["bw"] / 2).clip(0, 1)
    labels["y_max"] = (labels["y_centre"] + labels["bh"] / 2).clip(0, 1)
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"], "Unexpected columns found"

    return labels


def scale_labels(labels: pd.DataFrame, image_w: int, image_h: int):
    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    return labels


def labels_to_string(labels: pd.DataFrame):
    # Empty DataFrame means there was no YOLO output file or "reduce" removed all bboxes.
    # Means "No finding"
    if len(labels) == 0:
        return "14 1 0 0 1 1"
    assert labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"], "Unexpected columns found"
    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

### Process YOLO output files, and calculate the predictions

In [15]:
import ensemble_boxes

results_yolo_df = pd.DataFrame(columns=["image_id", "PredictionString"])
results_yolo_full_df = pd.DataFrame(columns=["image_id", "class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

weights = None
if yolo_versions_weighted:
    weights = [yolo_versions_weights[version] for version in INPUTS_YOLO_VERSIONS.keys()]
    # Make sure maximum is 1, and others are less than 1.
    weights = np.array(weights) / np.max(weights)

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    # Read YOLO output to pandas DataFrame.
    prediction_labels = [read_prediction_labels(
        version_data["path"] / "labels_pred" / f"{image_id}.txt"
    ) for version_data in INPUTS_YOLO_VERSIONS.values()]

    # Reduce bboxes.
    method = None if reduce_method == "None" else getattr(ensemble_boxes, reduce_method)
    prediction_labels = reduce_boxes(prediction_labels, weights=weights, method=method)

    if reduce_filter:
        # Reduce those that are in anomalous locations.
        prediction_labels = reduce_by_location_boundaries(prediction_labels)

        # Reduce those that should only have one class
        prediction_labels = reduce_max_x_per_class(prediction_labels)

    # Append all the boxes to full df for later analysis.
    prediction_labels_full = prediction_labels.copy()
    prediction_labels_full["image_id"] = image_id
    results_yolo_full_df = results_yolo_full_df.append(prediction_labels_full)

    # Scale them to the original size.
    # NOTE: dim0 and dim1 are reversed: y-axis, x-axis!
    prediction_labels = scale_labels(prediction_labels, image_dims["dim1"], image_dims["dim0"])

    # Convert resulting bboxes from DataFrame to string.
    prediction_str = labels_to_string(prediction_labels)

    results_yolo_df = results_yolo_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_yolo_df.sample(10, random_state=10)

,image_id,PredictionString
1779,9b41c13ac918adcc1261f750a1d61b40,11 0.336008 677 459 848 536 11 0.286166 1213 4...
341,202dc97c4b3ec7e50f4813456938500a,3 0.897488 706 1225 1553 1528 0 0.865153 962 6...
1276,711dc542c6c7e2b21889838cd050a810,7 0.188232 1346 990 1760 1560 7 0.112947 470 1...
1012,5ba0a669fd3430e9386758e838210b98,0 0.425730 1790 1182 2072 1472 7 0.233982 904 ...
470,2b2810284313c5547241b38668f67b2c,0 0.840630 1076 961 1356 1294 3 0.350491 749 1...
1755,990b0dcd83353894c4a2e73738cd6c25,3 0.187482 1025 1119 2150 1723 5 0.144485 1591...
2773,ec6c4e81b448275577d3e4a74acbe443,0 0.853516 1097 690 1318 941 3 0.402378 848 13...
1085,6100bc75e0a668c382d92d5eb9038076,3 0.679649 667 1104 1550 1401 0 0.214182 1027 ...
2055,b1a596318e7bfdcbb74044bf6d38c6ed,0 0.666640 960 588 1155 793 3 0.438478 641 126...
2585,de7aa2b6adfb9db39489c019f5f3a1a1,3 0.868517 684 1389 1807 1737 0 0.809727 1039 ...


## Merge with Binary classifier output

In [16]:
results_binary_df = pd.read_csv(INPUT_FOLDER_BINARY / "prediction.csv")
display(results_binary_df.head())

results_df = results_yolo_df.merge(results_binary_df, on="image_id")

if "confidence" in results_df.columns:
    # If confidence of "No finding" is very high, replace with "No finding"
    results_df.loc[results_df["confidence"] >= binary_normal_threshold, "PredictionString"] = "14 1 0 0 1 1"
    # If confidence of "No finding" is medium, add "No finding" with confidence to the object detector's bboxes.
    # Skip those that are already "No finding".
    results_df.loc[
            (results_df["confidence"] < binary_normal_threshold) & \
            (results_df["confidence"] > binary_abnormal_threshold) & \
            (~(results_df["PredictionString"] == "14 1 0 0 1 1")),
        "PredictionString"] += results_df["confidence"].apply(lambda prob: f" 14 {prob:.6f} 0 0 1 1")
    # If confidence of "No finding" is low, i.e. confidence of abnormality is high, do nothing.
else:
    # Just use binary classifier's suggestion as to which to set as "No finding"
    results_df.loc[results_df["class_name"] == "normal", "PredictionString"] = "14 1 0 0 1 1"

results_df = results_df[["image_id", "PredictionString"]]

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.head())

,image_id,class_name,confidence
0,980141b50ada8a624c9a1c34c91c95d8,abnormal,0.470578
1,f362fd880b0149646ccbb760be5f6354,normal,0.994644
2,0291515f5d14c34180a15712a55bf7bd,abnormal,0.089130
3,6cf8916303a07f3909297be170f21f7a,normal,0.970541
4,c6829051ecb281656c987fa2cfe5c706,abnormal,0.167964


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.636148 1259 585 1531 899 0 0.061068 1097 5...
2,008bdde2af2462e86fd373a445d0f4cd,0 0.854248 1429 825 1735 1184 3 0.828315 1096 ...
3,009bc039326338823ca3aa84381f17f1,3 0.801188 666 1057 1556 1353 0 0.604791 996 4...
4,00a2145de1886cb9eb88869c85d74080,3 0.862060 777 1293 1864 1645 0 0.806342 1123 ...


In [17]:
images_no_finding = results_df[results_df["PredictionString"] == "14 1 0 0 1 1"].image_id.values
results_yolo_full_df = results_yolo_full_df[~results_yolo_full_df.image_id.isin(images_no_finding)]

## Submission overview

Overall count of labels

In [18]:
display(results_df["PredictionString"].apply(lambda s: int(len(s.split(" ")) / 6)).sum())

204613

Counts of each class per image

In [19]:
class_counts_per_label = results_yolo_full_df.groupby("image_id").class_id.value_counts().unstack(level=-1).reset_index().fillna(0)
class_counts_per_label.index.rename("index")
class_counts_per_label.head(2)

class_id,image_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,004f33259ee4aef671c2b95d54e4be68,4.0,0.0,2.0,3.0,0.0,2.0,0.0,2.0,2.0,4.0,4.0,7.0,0.0,2.0
1,008bdde2af2462e86fd373a445d0f4cd,6.0,0.0,6.0,1.0,0.0,1.0,0.0,3.0,1.0,2.0,0.0,14.0,0.0,8.0


Average counts

In [20]:
class_counts_per_label.describe()

class_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000
mean,4.051115,0.895052,4.466014,1.827080,1.375204,6.947798,3.036433,8.244154,12.211528,18.353453,4.791735,21.764546,1.096792,20.631321
std,4.328160,2.422099,9.336230,1.811734,3.456642,10.187176,5.553898,11.149918,34.347095,28.591550,9.601563,14.904576,4.137383,26.747378
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,4.000000,0.000000,11.000000,0.000000,4.000000
50%,3.000000,0.000000,1.000000,1.000000,0.000000,3.000000,1.000000,4.000000,5.000000,9.000000,1.000000,19.000000,0.000000,11.000000
75%,5.000000,0.000000,5.000000,2.000000,1.000000,10.000000,4.000000,11.000000,11.000000,21.000000,4.000000,29.000000,1.000000,26.000000
max,51.000000,24.000000,147.000000,23.000000,34.000000,84.000000,54.000000,104.000000,752.000000,510.000000,87.000000,97.000000,102.000000,221.000000


Stats of bboxes per class

In [21]:
class_bbox_locations = results_yolo_full_df.copy()
class_bbox_locations["x_mid"] = (class_bbox_locations["x_min"] + class_bbox_locations["x_max"]) / 2
class_bbox_locations["y_mid"] = (class_bbox_locations["y_min"] + class_bbox_locations["y_max"]) / 2
class_bbox_locations = class_bbox_locations[["class_id", "x_mid", "y_mid", "conf"]]
class_bbox_locations.groupby("class_id").describe()

x_mid                      ...      conf                    
            count      mean       std  ...       50%       75%       max
class_id                               ...                              
0          7450.0  0.550277  0.084562  ...  0.003213  0.036862  0.881890
1          1646.0  0.532907  0.241146  ...  0.001980  0.004990  0.370657
2          8213.0  0.509653  0.214685  ...  0.001833  0.005683  0.555211
3          3360.0  0.558988  0.094491  ...  0.004846  0.257514  0.900391
4          2529.0  0.483679  0.235023  ...  0.002329  0.007147  0.389776
5         12777.0  0.512652  0.254287  ...  0.002634  0.010234  0.597494
6          5584.0  0.476353  0.235583  ...  0.002294  0.007634  0.404541
7         15161.0  0.483459  0.229956  ...  0.002397  0.008542  0.524376
8         22457.0  0.485730  0.237767  ...  0.001729  0.005015  0.793267
9         33752.0  0.475716  0.253512  ...  0.001645  0.004489  0.597567
10         8812.0  0.468414  0.336715  ...  0.002786  0.010666  0.722216
11        40025.0  0.499387  0.216380  ...  0.002970  0.010873  0.630194
12         2017.0  0.472300  0.272334  ...  0.001633  0.004133  0.662657
13        37941.0  0.469552  0.234831  ...  0.001978  0.006322  0.601609

[14 rows x 24 columns]

Total number of "No finding" before binary classifier filter

In [22]:
print((results_yolo_df["PredictionString"] == "14 1 0 0 1 1").sum())

0


Total number of "No finding"

In [23]:
(results_df["PredictionString"] == "14 1 0 0 1 1").sum()

1161

Total with class "No finding" appended

In [24]:
print(results_df["PredictionString"].str.contains(r" 14 \d\.\d+ 0 0 1 1").sum())

1728


## Submit to Kaggle

In [27]:
submission_message = " ***** ".join([
    version_notes,
    "; ".join([
        f"Reduce method: {reduce_method}",
        f"IoU {iou_threshold}",
        f"Conf {skip_box_threshold}",
        f"Sigma {sigma}",
        f"Binary Normal Thresh: {binary_normal_threshold}",
        f"Binary Abnormal Thresh: {binary_abnormal_threshold}",
    ]),
    f"Binary CLF: {binary_version} -{binary_notes}",
    "Ensemble YOLO: " + "; ".join(f"{yolo_version} - {version_data['version_notes']}" for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items()),
])
!echo "{submission_message}"

if SUBMIT:
    !kaggle competitions submit \
        vinbigdata-chest-xray-abnormalities-detection \
        -f {WORK_FOLDER}/submission.csv \
        -m "{submission_message}"
    !sleep 10
    !kaggle competitions submissions vinbigdata-chest-xray-abnormalities-detection

BBox reduction ***** Reduce method: weighted_boxes_fusion; IoU 0.6; Conf 0.01; Sigma 0.1; Binary Normal Thresh: 0.95; Binary Abnormal Thresh: 0.05 ***** Binary CLF: 7 - EfficientNetB3, include probabilities (for later double thresholding); image size ***** Ensemble YOLO: 18 - Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: exp_yolov5x_50epochs_WBFat0.4iou_640image_detect0.6iou0.01conf_fold0of5; 19 - Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf; 20 - Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x